In [1]:
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from bs4 import BeautifulSoup, SoupStrainer
import re
from urllib.parse import urljoin
import time
from datetime import datetime, timedelta
import statsmodels.api as sm
import statsmodels.formula.api as smf

C:\Users\tjsim\Anaconda\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


Scraping gamelogs for every team from seasons 2014-Present and saving each to a csv file 

In [2]:
years = list(range(2022,2024))


In [ ]:
url_start

In [29]:
for year in years:
    url = url_start.format(year)
    data = requests.get(url)
    time.sleep(3)
    
    with open('dal_gamelog/{}.html'.format(year), 'w+', encoding = 'utf-8') as nba:
        nba.write(data.text)

In [ ]:
#open 1 page example

with open('dal_gamelog/2017.html', encoding = 'utf-8') as nba:
    page = nba.read()

In [72]:
#soup = bs(page, 'html.parser')

In [73]:
#soup.find('tr', class_= 'over_header').decompose() 

In [74]:
#bos_gmlog_table = soup.find(id ='tgl_basic')

In [75]:
#bos_gmlog_17 = pd.read_html(str(bos_gmlog_table))[0]

In [76]:
#bos_gmlog_17

In [253]:
dfs = []
for year in years:
    with open('nba_gamelog/{}.html'.format(year), encoding = 'utf-8') as nba:
        page = nba.read()
        soup = bs(page, 'html.parser')
        soup.find('tr', class_= 'over_header').decompose() 
        soup.find('tr', class_= 'over_header thead').decompose()
        hou_gmlog_table = soup.find(id ='tgl_basic')
        hou_gmlogs_14_23 = pd.read_html(str(hou_gmlog_table))[0]
    
    dfs.append(hou_gmlogs_14_23)

In [255]:
hou = pd.concat(dfs)

In [256]:
hou = hou.rename(columns = {'Unnamed: 3': 'A/H',
                                        'Tm': 'Tm_PTS',
                                         'Opp.1': 'Opp_PTS',
                                         '3PA': 'Three_FGA',
                                         '3PA.1': 'OThree_FGA'})

In [257]:
hou['A/H'] = hou['A/H'].replace(np.nan, 'H')
hou = hou.replace({'@':'A'})

In [259]:
hou.to_csv('hou.csv')

In [2]:
for year in years:
    url = url_start.format(year)
    data = requests.get(url)
    
    with open('hou_gamelog/{}.html'.format(year), 'w+', encoding = 'utf-8') as nba:
        nba.write(data.text)


NameError: name 'years' is not defined

In [28]:
# Gets specified team gamelog

URL = 'https://www.basketball-reference.com/teams/IND/2023.html'        

r = requests.get(URL)

In [29]:
# parses html

soup = bs(r.text, 'html.parser')

In [30]:
pid = []

for href in soup.find_all('a', href=re.compile(r'.+gamelog.+')):
    playerid = href['href'][11:-14]
    if playerid:
        pid.append(playerid)

pid = list(set(pid))

In [31]:
print(pid)

['smithja04', 'duartch01', 'brownke03', 'turnemy01', 'hillge01', 'jacksis01', 'taylote01', 'brissos01', 'nembhan01', 'halibty01', 'queentr01', 'hieldbu01', 'mathube01', 'nworajo01', 'theisda01', 'johnsja01', 'mccontj01', 'nesmiaa01', 'bitadgo01']


In [32]:
year = 2023
letter = [0]
glogs = []

for p in pid:
    glog = pd.read_html(f'https://www.basketball-reference.com/players/{letter}/{p}/gamelog/{year}')[-1]
    glogs.append(glog)
    glog['player'] = p
    pd.concat(glogs)
    time.sleep(3)

In [34]:
a_glog_23 = pd.concat(glogs)

a_glog_23

,Rk,G,Date,Age,Tm,Unnamed: 5,Opp,Unnamed: 7,GS,MP,...,TRB,AST,STL,BLK,TOV,PF,PTS,GmSc,+/-,player
0,1,1,2022-10-19,22-217,IND,NaN,WAS,L (-7),1,22:03,...,8,1,0,1,0,2,16,11.2,0,smithja04
1,2,2,2022-10-21,22-219,IND,NaN,SAS,L (-3),1,6:47,...,2,0,0,0,1,1,2,-0.5,-11,smithja04
2,3,3,2022-10-22,22-220,IND,NaN,DET,W (+9),1,27:31,...,15,1,0,3,1,2,19,17.7,-1,smithja04
3,4,4,2022-10-24,22-222,IND,@,PHI,L (-14),1,25:22,...,10,1,1,1,1,2,17,15.8,-5,smithja04
4,5,5,2022-10-26,22-224,IND,@,CHI,L (-15),1,29:09,...,9,2,0,2,2,2,15,13.9,+5,smithja04
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
69,67,NaN,2023-03-11,23-234,ORL,NaN,MIA,W (+12),Did Not Play,Did Not Play,...,Did Not Play,Did Not Play,Did Not Play,Did Not Play,Did Not Play,Did Not Play,Did Not Play,Did Not Play,Did Not Play,bitadgo01
70,68,NaN,2023-03-14,23-237,ORL,@,SAS,L (-18),Did Not Play,Did Not Play,...,Did Not Play,Did Not Play,Did Not Play,Did Not Play,Did Not Play,Did Not Play,Did Not Play,Did Not Play,Did Not Play,bitadgo01
71,69,27,2023-03-16,23-239,ORL,@,PHO,L (-3),0,8:14,...,5,1,0,2,1,2,2,2.9,+9,bitadgo01
72,70,28,2023-03-18,23-241,ORL,@,LAC,W (+5),0,15:05,...,3,2,1,0,0,3,0,1.1,-8,bitadgo01


In [57]:
team = ['IND']
glog = []

for t in team:
    game_log = pd.read_html(f'https://www.basketball-reference.com/teams/{t}/2023/gamelog/', header = 1)[0]
    glog.append(game_log)
    glog = glog
    time.sleep(3)
    
a_glog_23 = pd.concat(glog)

In [58]:
a_glog_23

,Rk,G,Date,Unnamed: 3,Opp,W/L,Tm,Opp.1,FG,FGA,...,FT.1,FTA.1,FT%.1,ORB.1,TRB.1,AST.1,STL.1,BLK.1,TOV.1,PF.1
0,1,1,2022-10-19,NaN,WAS,L,107,114,39,97,...,19,24,.792,14,53,21,5,10,16,19
1,2,2,2022-10-21,NaN,SAS,L,134,137,51,104,...,32,44,.727,7,37,34,9,4,17,22
2,3,3,2022-10-22,NaN,DET,W,124,115,45,103,...,23,30,.767,17,53,26,5,3,13,20
3,4,4,2022-10-24,@,PHI,L,106,120,38,90,...,25,28,.893,7,39,25,8,7,7,20
4,5,5,2022-10-26,@,CHI,L,109,124,37,86,...,22,24,.917,9,44,34,10,5,17,25
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71,66,66,2023-03-06,NaN,PHI,L,143,147,53,90,...,35,38,.921,5,36,37,6,5,11,26
72,67,67,2023-03-09,NaN,HOU,W,134,125,48,100,...,21,29,.724,20,58,18,6,6,16,22
73,68,68,2023-03-11,@,DET,W,121,115,43,93,...,18,27,.667,12,42,25,7,7,12,24
74,69,69,2023-03-13,@,DET,L,97,117,36,86,...,7,10,.700,19,52,28,10,5,14,21


In [59]:
# Clean df 

In [60]:
a_glog_23 = a_glog_23.fillna(0)

In [61]:
a_glog_23 = a_glog_23[a_glog_23.Date != 'Date']

In [62]:
a_glog_23['Date'] = pd.to_datetime(a_glog_23['Date'], format='%Y-%m-%d')

In [63]:
a_glog_23['Days'] = a_glog_23['Date'].diff().dt.days

In [64]:
a_glog_23 = a_glog_23.rename(columns = {'Unnamed: 3': 'A/H',
                             'Unnamed: 7': 'W/L'})

In [65]:
a_glog_23['A/H'] = a_glog_23['A/H'].replace(0, 'H')


In [66]:
a_glog_23 = a_glog_23.replace({'@':'A'})

In [67]:
a_glog_23 = a_glog_23.rename(columns ={'A/H': 'away_home', 'W/L': 'Win_Loss', '+/-': 'plus_minus'})

In [68]:
ind_glog_23 = a_glog_23.fillna(0)

In [69]:
ind_glog_23.to_csv('pid_glog_23/ind_glog_23.csv')

Distribution

In [ ]:
eFG% = (FGM + 0.5 * 3PM) / FGA

In [ ]:
eFG%_constant = points = (efg% + (coeff * rest) * fga

In [9]:
team = ['NYK']
on_off = []

for t in team:
    On_Off = pd.read_html(f'https://www.basketball-reference.com/teams/{t}/2023/gamelog/')
    on_off.append(On_Off)
    on_off= on_off[0]
    time.sleep(3)

In [10]:
on_off = on_off[0]

In [11]:
on_off.columns = on_off.columns.droplevel(0)

In [12]:
nyk_oo_23 = on_off.fillna('')


In [70]:
lal_oo.columns

Index(['Player', 'Split', 'MP', 'eFG%', 'ORB%', 'DRB%', 'TRB%', 'AST%', 'STL%',
       'BLK%', 'TOV%', 'Pace', 'ORtg', 'eFG%.1', 'ORB%.1', 'DRB%.1', 'TRB%.1',
       'AST%.1', 'STL%.1', 'BLK%.1', 'TOV%.1', 'Pace.1', 'ORtg.1', 'eFG%.2',
       'ORB%.2', 'DRB%.2', 'TRB%.2', 'AST%.2', 'STL%.2', 'BLK%.2', 'TOV%.2',
       'Pace.2', 'ORtg.2'],
      dtype='object')

In [14]:
nyk_oo_23.to_csv('nba_oo_23/nyk_oo_23.csv')

In [159]:
uta_oo = pd.read_csv('nba_oo_23/uta_oo_23.csv').fillna('').drop('Unnamed: 0', axis = 1)#.rename(columns = {'eFG%.2': 'Opp_eFG%'})

In [160]:
dal_oo = pd.read_csv('nba_oo_23/dal_oo_23.csv').fillna('').drop('Unnamed: 0', axis = 1)#.rename(columns = {'eFG%.2': 'Opp_eFG%'})

In [161]:
brk_oo = pd.read_csv('nba_oo_23/brk_oo_23.csv').fillna('').drop('Unnamed: 0', axis = 1)#.rename(columns = {'eFG%.2': 'Opp_eFG%'})

In [162]:
hou_oo = pd.read_csv('nba_oo_23/hou_oo_23.csv').fillna('').drop('Unnamed: 0', axis = 1)#.rename(columns = {'eFG%.2': 'Opp_eFG%'})

In [67]:
lal_oo = pd.read_csv('nba_oo_23/lal_oo_23.csv').fillna('').drop('Unnamed: 0', axis = 1)#.rename(columns = {'eFG%.2': 'Opp_eFG%'})

In [68]:
nyk_oo = pd.read_csv('nba_oo_23/nyk_oo_23.csv').fillna('').drop('Unnamed: 0', axis = 1)#.rename(columns = {'eFG%.2': 'Opp_eFG%'})

In [66]:
lal_oo['Player'].value_counts()

NameError: name 'lal_oo' is not defined

In [52]:
nyk.columns

Index(['Player', 'Split', 'MP', 'eFG%', 'ORB%', 'DRB%', 'TRB%', 'AST%', 'STL%',
       'BLK%', 'TOV%', 'Pace', 'ORtg', 'eFG%.1', 'ORB%.1', 'DRB%.1', 'TRB%.1',
       'AST%.1', 'STL%.1', 'BLK%.1', 'TOV%.1', 'Pace.1', 'ORtg.1', 'eFG%.2',
       'ORB%.2', 'DRB%.2', 'TRB%.2', 'AST%.2', 'STL%.2', 'BLK%.2', 'TOV%.2',
       'Pace.2', 'ORtg.2'],
      dtype='object')

In [20]:
nyk = nyk_oo.loc[nyk_oo['Player'].isin(['Julius Randle', 'Mitchell Robinson', 'Immanuel Quickley', 'RJ Barrett', 'Quentin Grimes'])]

In [45]:
lal = lal_oo.loc[lal_oo['Player'].isin(['Jarred Vanderbilt','Malik Beasley','Troy Brown Jr.','Anthony Davis','D''Angelo Russell'])]

In [46]:
nyk['eFG%']

0     .542
8     .528
12    .522
16    .540
24    .550
Name: eFG%, dtype: object

In [47]:
lal['eFG%.1']

8     .533
16    .524
56    .511
60    .520
Name: eFG%.1, dtype: object

In [54]:
print(.542 * .533 / .544)
print(0.5310404411764706* 2.5 * 18)

0.5310404411764706
23.896819852941178


In [ ]:
#efg = efg * Opp_efga / nba_efg / fga
#drd = drb * Opp_drba / nba_drb / trb
#orb = orb * Opp_orba / nba_orb / trb
#tov = tov * Opp_tova / nba_tov / pos

In [ ]:
bos

In [114]:
#Grab every team, concat into one df --  can select amongst them

In [71]:
On_Off = 'https://www.basketball-reference.com/teams/CLE/2023/on-off/'

oo = requests.get(On_Off)

In [72]:
soup = bs(oo.text, 'html.parser')

In [73]:
on_off = pd.read_html(oo.content)

In [74]:
on_off = on_off[0] 

In [75]:
on_off.columns = on_off.columns.droplevel(0)

In [77]:
nyk_oo_23 = on_off.fillna('')

nyk_oo_23

,Player,Split,MP,eFG%,ORB%,DRB%,TRB%,AST%,STL%,BLK%,...,eFG%,ORB%,DRB%,TRB%,AST%,STL%,BLK%,TOV%,Pace,ORtg
0,Evan Mobley,On Court,2120,.557,24.5,78.3,51.5,61.1,7.5,9.0,...,+.015,+2.8,+2.8,+3.0,-0.8,-0.2,+0.4,-1.5,+0.2,+5.1
1,,Off Court,990,.555,22.6,75.5,49.7,58.7,7.2,7.0,...,+.038,-2.0,-2.0,-0.7,+6.8,-0.1,+0.6,-1.1,-0.2,+6.6
2,,On − Off,68%,+.002,+1.9,+2.8,+1.8,+2.4,+0.3,+2.0,...,-.023,+4.8,+4.8,+3.7,-7.6,-0.1,-0.2,-0.4,+0.4,-1.5
3,,,,,,,,,,,...,,,,,,,,,,
4,Darius Garland,On Court,1898,.565,24.3,77.2,51.0,60.5,7.6,8.2,...,+.026,+1.5,+1.5,+2.0,+0.5,+0.6,-0.4,-2.4,+1.0,+6.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58,,On − Off,1%,+.013,-11.5,-12.8,-16.9,+6.4,+6.3,+0.0,...,-.129,-24.2,-24.2,-33.8,+19.3,+8.7,+0.6,-10.4,-1.8,-22.5
59,Player,Split,MP,eFG%,ORB%,DRB%,TRB%,AST%,STL%,BLK%,...,eFG%,ORB%,DRB%,TRB%,AST%,STL%,BLK%,TOV%,Pace,ORtg
60,Isaiah Mobley,On Court,26,.643,17.6,76.5,56.9,50.0,6.3,8.7,...,+.302,-5.9,-5.9,+13.7,-14.3,+4.2,+5.1,0.0,0.0,+52.1
61,,Off Court,3084,.556,23.9,77.4,50.8,60.5,7.4,8.3,...,+.020,+1.3,+1.3,+1.7,+1.8,-0.2,+0.3,-1.4,+0.1,+5.2


In [78]:
nyk_oo_23.to_csv('nba_oo_23/nyk_oo_23.csv')

Cleaning and creating East/West/League gamelog dataframes. Adding datetime difference columns to League dataframe.  

In [175]:
nba_east_gamelog_14_23 = pd.concat(
    map(pd.read_csv,['nba_east/atl.csv', 'nba_east/bos.csv', 'nba_east/brk.csv', 'nba_east/chi.csv',
                    'nba_east/cho.csv', 'nba_east/cle.csv', 'nba_east/det.csv', 'nba_east/ind.csv',
                    'nba_east/mia.csv', 'nba_east/mil.csv', 'nba_east/nyk.csv', 'nba_east/orl.csv',
                    'nba_east/phi.csv', 'nba_east/tor.csv', 'nba_east/was.csv']), ignore_index = True)

In [176]:
nba_east_gamelog_14_23 

,Unnamed: 0,Tm,Rk,G,Date,A/H,Opp,W/L,Tm_PTS,Opp_PTS,...,FT.1,FTA.1,FT%.1,ORB.1,TRB.1,AST.1,STL.1,BLK.1,TOV.1,PF.1
0,0,ATL,1,1,2014-10-29,A,TOR,L,102,109,...,27,33,.818,16,48,26,13,9,9,22
1,1,ATL,2,2,2014-11-01,H,IND,W,102,92,...,18,21,.857,11,44,25,5,5,18,26
2,2,ATL,3,3,2014-11-05,A,SAS,L,92,94,...,27,38,.711,11,50,25,7,9,19,15
3,3,ATL,4,4,2014-11-07,A,CHO,L,119,122,...,20,27,.741,11,51,31,6,7,19,30
4,4,ATL,5,5,2014-11-08,H,NYK,W,103,96,...,8,11,.727,13,44,26,2,6,15,29
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10878,723,WAS,35,35,2022-12-27,H,PHI,W,116,111,...,27,31,.871,10,39,21,8,10,15,24
10879,724,WAS,36,36,2022-12-28,H,PHO,W,127,102,...,11,16,.688,11,30,25,7,4,11,21
10880,725,WAS,37,37,2022-12-30,A,ORL,W,119,100,...,11,16,.688,8,35,24,8,2,11,18
10881,726,WAS,38,38,2023-01-01,A,MIL,W,118,95,...,14,15,.933,14,44,22,11,3,16,15


In [174]:
nba_east_gamelog_14_23.to_csv('nba_east_gamelog_14_23.csv') 

In [168]:
nba_west_gamelog_14_23 = pd.concat(
    map(pd.read_csv,['nba_west/dal.csv', 'nba_west/den.csv', 'nba_west/gsw.csv', 'nba_west/hou.csv',
                     'nba_west/lac.csv', 'nba_west/lal.csv', 'nba_west/mem.csv', 'nba_west/mem.csv',
                     'nba_west/min.csv', 'nba_west/nop.csv', 'nba_west/okc.csv', 'nba_west/pho.csv',
                     'nba_west/por.csv', 'nba_west/sac.csv', 'nba_west/sas.csv', 'nba_west/uta.csv']), ignore_index = True)

In [177]:
nba_west_gamelog_14_23

,Unnamed: 0,Tm,Rk,G,Date,A/H,Opp,W/L,Tm_PTS,Opp_PTS,...,FT.1,FTA.1,FT%.1,ORB.1,TRB.1,AST.1,STL.1,BLK.1,TOV.1,PF.1
0,0,DAL,1,1,2014-10-28,A,SAS,L,100,101,...,13,16,.813,9,38,23,5,3,20,20
1,1,DAL,2,2,2014-10-30,H,UTA,W,120,102,...,16,18,.889,15,40,22,6,0,16,21
2,2,DAL,3,3,2014-11-01,A,NOP,W,109,104,...,21,28,.750,14,47,24,8,4,13,19
3,3,DAL,4,4,2014-11-03,H,BOS,W,118,113,...,12,16,.750,22,49,29,9,6,17,21
4,4,DAL,5,5,2014-11-06,A,POR,L,87,108,...,16,19,.842,12,53,25,3,6,17,21
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11629,727,UTA,39,39,2022-12-31,H,MIA,L,123,126,...,23,30,.767,10,37,32,10,1,12,32
11630,728,UTA,40,40,2023-01-03,H,SAC,L,115,117,...,16,18,.889,7,40,23,7,4,17,26
11631,729,UTA,NaN,NaN,NaN,H,NaN,NaN,NaN,NaN,...,Opponent,Opponent,Opponent,Opponent,Opponent,Opponent,Opponent,Opponent,Opponent,Opponent
11632,730,UTA,Rk,G,Date,H,Opp,W/L,Tm,Opp,...,FT,FTA,FT%,ORB,TRB,AST,STL,BLK,TOV,PF


In [6]:
nba_west_gamelog_14_23.to_csv('nba_west_gamelog_14_23.csv')

In [106]:
nba_league_gmlg_14_23 = pd.concat(
    map(pd.read_csv,['nba_east/nba_east_gamelog_14_23.csv', 'nba_west/nba_west_gamelog_14_23.csv']), ignore_index = True)

In [167]:
nba_east_gamelog_14_23

,Unnamed: 0,Tm,Rk,G,Date,A/H,Opp,W/L,Tm_PTS,Opp_PTS,...,FT.1,FTA.1,FT%.1,ORB.1,TRB.1,AST.1,STL.1,BLK.1,TOV.1,PF.1
0,0,ATL,1,1,2014-10-29,A,TOR,L,102,109,...,27,33,.818,16,48,26,13,9,9,22
1,1,ATL,2,2,2014-11-01,H,IND,W,102,92,...,18,21,.857,11,44,25,5,5,18,26
2,2,ATL,3,3,2014-11-05,A,SAS,L,92,94,...,27,38,.711,11,50,25,7,9,19,15
3,3,ATL,4,4,2014-11-07,A,CHO,L,119,122,...,20,27,.741,11,51,31,6,7,19,30
4,4,ATL,5,5,2014-11-08,H,NYK,W,103,96,...,8,11,.727,13,44,26,2,6,15,29
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10878,723,WAS,35,35,2022-12-27,H,PHI,W,116,111,...,27,31,.871,10,39,21,8,10,15,24
10879,724,WAS,36,36,2022-12-28,H,PHO,W,127,102,...,11,16,.688,11,30,25,7,4,11,21
10880,725,WAS,37,37,2022-12-30,A,ORL,W,119,100,...,11,16,.688,8,35,24,8,2,11,18
10881,726,WAS,38,38,2023-01-01,A,MIL,W,118,95,...,14,15,.933,14,44,22,11,3,16,15


In [132]:
nba_league_gmlg_14_23 = nba_league_gmlg_14_23[nba_league_gmlg_14_23.Date != 'Date']
nba_league_gmlg_14_23  = nba_league_gmlg_14_23.dropna(axis=0)
nba_league_gmlg_14_23 = nba_league_gmlg_14_23.drop(['Unnamed: 0.1'], axis =1)


KeyError: "['Unnamed: 0.1'] not found in axis"

In [133]:
nba_league_gmlg_14_23

,Tm,Rk,G,Date,A/H,Opp,W/L,Tm_PTS,Opp_PTS,FG,...,FT.1,FTA.1,FT%.1,ORB.1,TRB.1,AST.1,STL.1,BLK.1,TOV.1,PF.1
0,ATL,1,1,2014-10-29,A,TOR,L,102,109,40,...,27,33,.818,16,48,26,13,9,9,22
1,ATL,2,2,2014-11-01,H,IND,W,102,92,35,...,18,21,.857,11,44,25,5,5,18,26
2,ATL,3,3,2014-11-05,A,SAS,L,92,94,38,...,27,38,.711,11,50,25,7,9,19,15
3,ATL,4,4,2014-11-07,A,CHO,L,119,122,43,...,20,27,.741,11,51,31,6,7,19,30
4,ATL,5,5,2014-11-08,H,NYK,W,103,96,33,...,8,11,.727,13,44,26,2,6,15,29
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22510,UTA,37,37,2022-12-28,A,GSW,L,107,112,37,...,22,27,.815,10,51,22,6,3,12,26
22511,UTA,38,38,2022-12-30,A,SAC,L,125,126,46,...,14,19,.737,10,38,32,5,3,15,21
22512,UTA,39,39,2022-12-31,H,MIA,L,123,126,35,...,23,30,.767,10,37,32,10,1,12,32
22513,UTA,40,40,2023-01-03,H,SAC,L,115,117,38,...,16,18,.889,7,40,23,7,4,17,26


In [152]:
nba_league_gmlg_14_23['Date'] = pd.to_datetime(nba_league_gmlg_14_23['Date'], format='%Y-%m-%d')
nba_league_gmlg_14_23['Days'] = nba_league_gmlg_14_23.groupby('Tm')['Date'].diff().dt.days
nba_league_gmlg_14_23['Hours'] = nba_league_gmlg_14_23.groupby('Tm')['Date'].diff().dt.days.multiply(24)
nba_league_gmlg_14_23['Minutes'] = nba_league_gmlg_14_23.groupby('Tm')['Date'].diff().dt.days.multiply(60)

In [157]:
nba_league_gmlg_14_23 = nba_league_gmlg_14_23.fillna(0)

In [162]:
nba_league_gmlg_14_23 = nba_league_gmlg_14_23.rename(columns={'3P':'TFG','Three_FGA':'TFGA','3P%':'TP%',
                                      'FG.1':'Opp_FG', 'FGA.1': 'Opp_FGA',
                                      'FG%.1':'Opp_FG%', '3P.1':'Opp_TFG',
                                      'OThree_FGA':'Opp_TFGA', '3P%.1':'Opp_TP%',
                                      'FT.1':'Opp_FT', 'FTA.1':'Opp_FTA',
                                      'FT%.1':'Opp_FT%', 'ORB.1':'Opp_ORB',
                                      'TRB.1':'Opp_TRB', 'AST.1':'Opp_AST',
                                      'STL.1':'Opp_STL','BLK.1':'Opp_BLK', 
                                      'TOV.1':'Opp_TOV','PF.1':'Opp_PF'})
                            
                                      

In [163]:
nba_league_gmlg_14_23

,Tm,Rk,G,Date,A/H,Opp,W/L,Tm_PTS,Opp_PTS,FG,...,Opp_TRB,Opp_AST,Opp_STL,Opp_BLK,Opp_TOV,Opp_PF,Rest,Days,Hours,Minutes
0,ATL,1,1,2014-10-29,A,TOR,L,102,109,40,...,48,26,13,9,9,22,0.0,0.0,0.0,0.0
1,ATL,2,2,2014-11-01,H,IND,W,102,92,35,...,44,25,5,5,18,26,3.0,3.0,72.0,180.0
2,ATL,3,3,2014-11-05,A,SAS,L,92,94,38,...,50,25,7,9,19,15,4.0,4.0,96.0,240.0
3,ATL,4,4,2014-11-07,A,CHO,L,119,122,43,...,51,31,6,7,19,30,2.0,2.0,48.0,120.0
4,ATL,5,5,2014-11-08,H,NYK,W,103,96,33,...,44,26,2,6,15,29,1.0,1.0,24.0,60.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22510,UTA,37,37,2022-12-28,A,GSW,L,107,112,37,...,51,22,6,3,12,26,2.0,2.0,48.0,120.0
22511,UTA,38,38,2022-12-30,A,SAC,L,125,126,46,...,38,32,5,3,15,21,2.0,2.0,48.0,120.0
22512,UTA,39,39,2022-12-31,H,MIA,L,123,126,35,...,37,32,10,1,12,32,1.0,1.0,24.0,60.0
22513,UTA,40,40,2023-01-03,H,SAC,L,115,117,38,...,40,23,7,4,17,26,3.0,3.0,72.0,180.0


In [164]:
nba_league_gmlg_14_23.to_csv('nba_league_gmlg_14_23.csv')


In [4]:
pd.read_csv('west_player_glog_22_23/dal_glog_22.csv')

,Unnamed: 0,Rk,G,Date,Age,Tm,A/H,Opp,W/L,GS,...,AST,STL,BLK,TOV,PF,PTS,GmSc,+/-,player,Days
0,0,1,1.0,2021-10-21,22-235,DAL,A,ATL,L (-26),1,...,7,1,0,5,1,18,12.3,-24,doncilu01,0.0
1,1,2,2.0,2021-10-23,22-237,DAL,A,TOR,W (+8),1,...,12,0,1,5,3,27,19.4,+1,doncilu01,2.0
2,2,3,3.0,2021-10-26,22-240,DAL,H,HOU,W (+10),1,...,7,2,1,4,3,26,19.9,+15,doncilu01,3.0
3,3,4,4.0,2021-10-28,22-242,DAL,H,SAS,W (+5),1,...,5,0,1,7,3,25,11.8,-19,doncilu01,2.0
4,4,5,5.0,2021-10-29,22-243,DAL,A,DEN,L (-31),1,...,4,2,0,2,1,16,9.6,-32,doncilu01,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1816,80,78,76.0,2022-04-01,28-332,DAL,A,WAS,L (-32),1,...,1,1,0,1,3,8,5.5,-29,finnedo01,2.0
1817,81,79,77.0,2022-04-03,28-334,DAL,A,MIL,W (+6),1,...,6,2,1,0,2,11,12.7,+9,finnedo01,2.0
1818,82,80,78.0,2022-04-06,28-337,DAL,A,DET,W (+18),1,...,3,1,0,1,2,13,12.7,+19,finnedo01,3.0
1819,84,81,79.0,2022-04-08,28-339,DAL,H,POR,W (+50),1,...,1,2,1,0,1,15,18.0,+36,finnedo01,2.0


In [7]:
nba_west_player_glog = pd.concat(
    map(pd.read_csv,['west_player_glog_22_23/dal_glog_22.csv',
                     'west_player_glog_22_23/dal_glog_23.csv',
                     'west_player_glog_22_23/den_glog_22.csv',
                     'west_player_glog_22_23/den_glog_23.csv',
                     'west_player_glog_22_23/det_glog_22.csv',
                     'west_player_glog_22_23/det_glog_23.csv',
                     'west_player_glog_22_23/gsw_glog_22.csv',
                     'west_player_glog_22_23/gsw_glog_23.csv',
                     'west_player_glog_22_23/hou_glog_22.csv',
                     'west_player_glog_22_23/hou_glog_23.csv',
                     'west_player_glog_22_23/lac_glog_22.csv',
                     'west_player_glog_22_23/lac_glog_23.csv',
                     'west_player_glog_22_23/lal_glog_22.csv',
                     'west_player_glog_22_23/lal_glog_23.csv',
                     'west_player_glog_22_23/mem_glog_22.csv',
                     'west_player_glog_22_23/mem_glog_23.csv',
                     'west_player_glog_22_23/min_glog_22.csv',
                     'west_player_glog_22_23/min_glog_23.csv',
                     'west_player_glog_22_23/nop_glog_22.csv',
                     'west_player_glog_22_23/nop_glog_23.csv',
                     'west_player_glog_22_23/okc_glog_22.csv',
                     'west_player_glog_22_23/okc_glog_23.csv',
                     'west_player_glog_22_23/pho_glog_22.csv',
                     'west_player_glog_22_23/pho_glog_23.csv',
                     'west_player_glog_22_23/por_glog_22.csv',
                     'west_player_glog_22_23/por_glog_23.csv',
                     'west_player_glog_22_23/sac_glog_22.csv',
                     'west_player_glog_22_23/sac_glog_23.csv',
                     'west_player_glog_22_23/sas_glog_22.csv',
                     'west_player_glog_22_23/sas_glog_23.csv',
                     'west_player_glog_22_23/tor_glog_22.csv',
                     'west_player_glog_22_23/tor_glog_23.csv',
                     'west_player_glog_22_23/uta_glog_22.csv',
                     'west_player_glog_22_23/uta_glog_22.csv'
                    ]), ignore_index = True)

In [10]:
nba_west_player_glog.to_csv('nba_wst_pglog_22_23.csv')


In [3]:
nba_east_player_glog = pd.concat(
    map(pd.read_csv,['east_player_glog_22_23/atl_glog_22.csv',
                    'east_player_glog_22_23/atl_glog_23.csv',
                    'east_player_glog_22_23/bos_glog_22.csv',
                    'east_player_glog_22_23/bos_glog_23.csv',
                    'east_player_glog_22_23/brk_glog_22.csv',
                    'east_player_glog_22_23/brk_glog_23.csv',
                    'east_player_glog_22_23/chi_glog_22.csv',
                    'east_player_glog_22_23/chi_glog_23.csv',
                    'east_player_glog_22_23/cho_glog_22.csv',
                    'east_player_glog_22_23/cho_glog_23.csv',
                    'east_player_glog_22_23/cle_glog_22.csv',
                    'east_player_glog_22_23/cle_glog_23.csv',
                    'east_player_glog_22_23/ind_glog_22.csv',
                    'east_player_glog_22_23/ind_glog_23.csv',
                    'east_player_glog_22_23/mia_glog_22.csv',
                    'east_player_glog_22_23/mia_glog_23.csv',
                    'east_player_glog_22_23/mil_glog_22.csv',
                    'east_player_glog_22_23/mil_glog_23.csv',
                    'east_player_glog_22_23/nyk_glog_22.csv',
                    'east_player_glog_22_23/nyk_glog_23.csv',
                    'east_player_glog_22_23/orl_glog_22.csv',
                    'east_player_glog_22_23/orl_glog_23.csv',
                    'east_player_glog_22_23/phi_glog_22.csv',
                    'east_player_glog_22_23/phi_glog_23.csv',
                    'east_player_glog_22_23/was_glog_22.csv',
                    'east_player_glog_22_23/was_glog_23.csv']), ignore_index = True)

In [9]:
nba_east_player_glog.to_csv('nba_est_pglog_22_23.csv')


In [16]:
nba_player_glog_22_23 = pd.concat(
    map(pd.read_csv,['nba_est_pglog_22_23.csv','nba_wst_pglog_22_23.csv']), ignore_index = True)

In [20]:
nba_player_glog_22_23.to_csv('nba_pglog_22_23.csv')

In [4]:
base_url = "https://www.basketball-reference.com"

In [5]:
teams = ["ATL", "BOS", "BRK", "CHI", "CLE", "DAL", "DEN", "DET", "GSW", "HOU", 
         "IND", "LAC", "LAL", "MEM", "MIA", "MIL", "MIN", "NOP", "NYK", "OKC", 
         "ORL", "PHI", "PHX", "POR", "SAC", "SAS", "TOR", "UTA", "WAS"]

In [9]:
lineup = []
for team in teams:
    lineups = pd.read_html(f'https://www.basketball-reference.com/teams/{team}/2023_start.html')[-1]
    lineup.append(lineups)
    pd.concat(lineup)
    time.sleep(4)
            

HTTPError: HTTP Error 404: Not Found

In [ ]:
lineup.csv(

In [ ]:

    soup = BeautifulSoup(page.content, 'html.parser')

page = requests.get(url)
